In [1]:
# Célula 1 – Inicialização, Login e Configuração do Navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

def iniciar_navegador():
    try:
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_experimental_option("detach", True)
        
        driver_manager = ChromeDriverManager()
        driver_path = driver_manager.install()
        
        driver_dir = os.path.dirname(driver_path)
        chromedriver_path = os.path.join(driver_dir, "chromedriver.exe")
        if not os.path.exists(chromedriver_path):
            print(f"Procurando chromedriver.exe em: {driver_dir}")
            for file in os.listdir(driver_dir):
                if "chromedriver.exe" in file.lower():
                    chromedriver_path = os.path.join(driver_dir, file)
                    print(f"Encontrado em: {chromedriver_path}")
                    break
        
        print(f"Usando ChromeDriver em: {chromedriver_path}")
        service = Service(executable_path=chromedriver_path)
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.implicitly_wait(3)  # Aumentado para 3 segundos
        driver.maximize_window()
        return driver
        
    except Exception as e:
        print(f"Erro ao iniciar Chrome: {e}")
        return None

def login_attus(driver):
    try:
        print("Acessando o site do Attus...")
        driver.get("https://pgeweb.sp.gov.br/app/mesa")
        actions = ActionChains(driver)
        wait = WebDriverWait(driver, 10)  # Aumentando timeout para 10 segundos
        
        # Login com email
        print("Esperando pela tela de login...")
        time.sleep(2)  # Aumentado para 2 segundos
        email_xpath = "/html/body/pd-root/pd-auth-container/div/div/mat-card/mat-card-content/pd-auth-login/pd-auth-login-username/form/div[1]/pd-form-text/mat-form-field/div[1]/div[2]/div/input"
        email_field = wait.until(EC.element_to_be_clickable((By.XPATH, email_xpath)))
        print("Campo de email encontrado, inserindo email...")
        email_field.clear()
        time.sleep(0.5)
        email_field.send_keys("gcabreu@sp.gov.br")
        print("Email inserido.")
        
        avancar_xpath = "/html/body/pd-root/pd-auth-container/div/div/mat-card/mat-card-content/pd-auth-login/pd-auth-login-username/form/div[2]/pd-button-primary/button/span[1]"
        avancar_button = wait.until(EC.element_to_be_clickable((By.XPATH, avancar_xpath)))
        print("Botão Avançar encontrado, clicando...")
        time.sleep(0.5)
        actions.move_to_element(avancar_button).click().perform()
        print("Clicado no botão Avançar.")
        
        # Login com senha
        print("Esperando pela tela de senha...")
        time.sleep(2)  # Aumentado para 2 segundos
        senha_xpath = "/html/body/pd-root/pd-auth-container/div/div/mat-card/mat-card-content/pd-auth-login/pd-auth-login-senha/form/div[2]/pd-form-text/mat-form-field/div[1]/div[2]/div/input"
        senha_field = wait.until(EC.element_to_be_clickable((By.XPATH, senha_xpath)))
        print("Campo de senha encontrado, inserindo senha...")
        senha_field.clear()
        time.sleep(0.5)
        senha_field.send_keys("gCA280292")
        print("Senha inserida.")
        
        entrar_xpath = "/html/body/pd-root/pd-auth-container/div/div/mat-card/mat-card-content/pd-auth-login/pd-auth-login-senha/form/div[3]/pd-button-primary/button/span[1]"
        entrar_button = wait.until(EC.element_to_be_clickable((By.XPATH, entrar_xpath)))
        print("Botão Entrar encontrado, clicando...")
        time.sleep(0.5)
        actions.move_to_element(entrar_button).click().perform()
        print("Clicado no botão Entrar.")
        
        # Aguardando login completo
        print("Aguardando conclusão do login...")
        time.sleep(5)  # Aumentado para 5 segundos para dar tempo de carregar após o login
        
        print("Login concluído com sucesso!")
        return driver
        
    except Exception as e:
        print(f"Erro no login: {e}")
        return None

In [2]:
# Célula 2 – Funções de Manipulação de Grupamento e Navegação
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

def encontrar_intimacao_transito(driver):
    def scroll_ate_encontrar():
        last_height = driver.execute_script("return document.body.scrollHeight")
        max_tentativas = 2
        for _ in range(max_tentativas):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.5)
            elements = driver.find_elements(By.TAG_NAME, "pd-demanda-agrupada")
            for element in elements:
                if "Intimação - Trânsito em julgado" in element.text:
                    return element
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        return None

    def rolar_pagina_grupo():
        print("\nRealizando rolagens para carregar mais intimações...")
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(0.5)
        for i in range(4):  # Alterado de 2 para 4 iterações
            actions = ActionChains(driver)
            for _ in range(5):
                actions.send_keys(Keys.PAGE_DOWN)
                time.sleep(0.2)
            actions.perform()
            time.sleep(1)
            print(f"Rolagem {i+1} completada")
        print("\nVoltando ao topo...")
        primeiro_elemento = driver.find_element(By.CSS_SELECTOR, "pd-demanda-mesa pd-demanda:first-child")
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", primeiro_elemento)
        time.sleep(0.5)

    try:
        print("Buscando grupamento de intimações...")
        wait = WebDriverWait(driver, 5)
        elementos = driver.find_elements(By.TAG_NAME, "pd-demanda-agrupada")
        elemento_encontrado = None
        for elemento in elementos:
            if "Intimação - Trânsito em julgado" in elemento.text:
                elemento_encontrado = elemento
                print("Grupamento encontrado sem necessidade de rolagem")
                break
        if not elemento_encontrado:
            print("Iniciando busca com rolagem...")
            elemento_encontrado = scroll_ate_encontrar()
            if elemento_encontrado:
                print("Grupamento encontrado após rolagem")
        if elemento_encontrado:
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", elemento_encontrado)
            time.sleep(0.5)
            try:
                elemento_encontrado.click()
            except:
                try:
                    actions = ActionChains(driver)
                    actions.move_to_element(elemento_encontrado).click().perform()
                except:
                    driver.execute_script("arguments[0].click();", elemento_encontrado)
            time.sleep(0.5)
            try:
                nao_alocados = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'Não alocados')]")))
                nao_alocados.click()
            except:
                nao_alocados = wait.until(EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'Não alocados')]")))
                driver.execute_script("arguments[0].click();", nao_alocados)
            rolar_pagina_grupo()
            return elemento_encontrado
        print("Grupamento não encontrado após todas as tentativas")
        return None
    except Exception as e:
        print(f"Erro ao buscar intimação: {e}")
        return None

In [3]:
# Célula 3 – Função para Clicar na Primeira Intimação Não Processada
import re
from selenium.webdriver.support import expected_conditions as EC

def clicar_primeira_intimacao_nao_processada(driver, intimacoes_processadas=set()):
    try:
        intimacoes = WebDriverWait(driver, 2).until(
            EC.presence_of_all_elements_located((
                By.CSS_SELECTOR, 
                "pd-demanda-mesa pd-demanda pd-card-deprecated"
            ))
        )
        if intimacoes:
            print(f"Encontradas {len(intimacoes)} intimações no grupo")
            for intimacao in intimacoes:
                texto_intimacao = intimacao.text.strip()
                numeros = re.findall(r'\d{7}-\d{2}\.\d{4}\.\d\.\d{2}\.\d{4}', texto_intimacao)
                id_intimacao = numeros[0] if numeros else texto_intimacao
                if id_intimacao and id_intimacao not in intimacoes_processadas:
                    print(f"Processo a ser clicado: {id_intimacao}")
                    # Rola a intimação para o centro da tela
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", intimacao)
                    time.sleep(0.5)
                    driver.execute_script("arguments[0].click();", intimacao)
                    intimacoes_processadas.add(id_intimacao)
                    return True, intimacoes_processadas
            return False, intimacoes_processadas
        return False, intimacoes_processadas
    except Exception as e:
        print(f"Erro ao clicar na próxima intimação: {e}")
        return False, intimacoes_processadas

In [4]:
# Célula 4 – Função para Exibir Processos
def exibir_processos(driver):
    try:
        window_after = driver.window_handles[-1]
        driver.switch_to.window(window_after)
        xpath_botao = "/html/body/pd-root/pd-app-environment/ng-component/div/main/div/button/span[4]"
        botao = driver.find_element(By.XPATH, xpath_botao)
        driver.execute_script("arguments[0].click();", botao)
        return True
    except Exception as e:
        try:
            botao = driver.find_element(By.XPATH, "//button[contains(.,'Exibir')]")
            driver.execute_script("arguments[0].click();", botao)
            return True
        except Exception as e:
            print(f"Erro ao clicar em 'Exibir processos': {e}")
            return False

In [5]:
# Célula 5 – Função para Processar Intimações (Histórico e Andamentos)
def processar_intimacoes(driver, intimacoes_processadas=set()):
    def buscar_siglas_no_historico():
        try:
            historico = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "pd-detalhes-andamento-historico"))
            )
            time.sleep(0.5)
            elementos_texto = historico.find_elements(By.XPATH, ".//*[text()]")
            if not elementos_texto:
                elementos_texto = historico.find_elements(By.CSS_SELECTOR, "div, span, p")
            print("\nBuscando siglas no histórico...")
            for elemento in elementos_texto:
                texto = elemento.text.upper()
                print(f"Texto encontrado: {texto}")
                for sigla in ['PPL', 'PPH', 'ISL']:
                    if sigla in texto:
                        print(f"Sigla {sigla} encontrada!")
                        return True
            texto_completo = historico.text.upper()
            for sigla in ['PPL', 'PPH', 'ISL']:
                if sigla in texto_completo:
                    print(f"Sigla {sigla} encontrada no texto completo!")
                    return True
            print("Nenhuma sigla encontrada no histórico")
            return False
        except Exception as e:
            print(f"Erro na busca de siglas: {e}")
            return False

    def fechar_aba_e_voltar():
        if len(driver.window_handles) > 1:
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            return True
        return False

    def processar_andamento(andamento):
        try:
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", andamento)
            botoes = andamento.find_elements(By.TAG_NAME, "button")
            if len(botoes) >= 2:
                driver.execute_script("arguments[0].click();", botoes[1])
                siglas_encontradas = buscar_siglas_no_historico()
                if not siglas_encontradas:
                    try:
                        ok_button = WebDriverWait(driver, 2).until(
                            EC.element_to_be_clickable((By.XPATH,
                                "//button[contains(@class, 'mat-primary')]//span[contains(text(), 'OK')]"
                            ))
                        )
                        driver.execute_script("arguments[0].click();", ok_button)
                    except:
                        pass
                return siglas_encontradas
            return False
        except Exception as e:
            print(f"Erro ao processar andamento: {e}")
            return False

    def processar_andamentos():
        try:
            time.sleep(1)
            # Definindo a ordem das intimações a serem processadas:
            tipos_prioridade = ["Intimação de Acórdão", "Intimação de Sentença", "Intimação - Não Classificada"]
            for tipo in tipos_prioridade:
                xpath = f"//span[contains(text(), '{tipo}')]/ancestor::pd-pasta-digital-andamento-card"
                andamentos = driver.find_elements(By.XPATH, xpath)
                if andamentos:
                    print(f"Processando andamentos do tipo: {tipo}")
                    for andamento in andamentos:
                        if processar_andamento(andamento):
                            return True
            return False
        except Exception as e:
            print(f"Erro ao processar andamentos: {e}")
            return False

    try:
        if processar_andamentos():
            return True
        fechar_aba_e_voltar()
        return False
    except Exception as e:
        print(f"Erro geral: {e}")
        return False

In [6]:
# Célula 6 – Função para Anotar Processos
def anota_ja_analis(driver):
    def clicar_elemento(xpath, descricao, timeout=3):
        try:
            time.sleep(0.25)
            wait = WebDriverWait(driver, timeout)
            elemento = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
            driver.execute_script("arguments[0].click();", elemento)
            time.sleep(0.25)
            print(f"✓ {descricao}")
            return True
        except Exception as e:
            print(f"✗ Erro ao {descricao.lower()}")
            return False

    try:
        try:
            time.sleep(0.25)
            clicar_elemento("//button[contains(@class, 'mat-primary')]//span[contains(text(), 'OK')]", "Fechou modal de detalhes")
        except:
            pass
        if not clicar_elemento("/html/body/pd-root/pd-app-environment/ng-component/div/main/div/button/span[4]", "Clicou em Exibir Processos"):
            return False
        if not clicar_elemento("/html/body/pd-root/pd-app-environment/ng-component/div/main/pd-pasta-digital-demandas-container/div/div/div/pd-pasta-digital-demanda-card/div/article/div[1]/header/div[1]/pd-action-buttons/div/div/div/button[2]", "Clicou em Incluir Observação"):
            return False
        try:
            time.sleep(0.25)
            wait = WebDriverWait(driver, 3)
            textarea = wait.until(EC.presence_of_element_located((By.XPATH, "//textarea")))
            textarea.clear()
            time.sleep(0.25)
            textarea.send_keys("Autos já analisados - GCA")
            time.sleep(0.25)
            print("✓ Inseriu texto da observação")
        except Exception as e:
            print(f"✗ Erro ao inserir texto")
            return False
        if not clicar_elemento("/html/body/div[3]/div[2]/div/ng-component/pd-modal-container/mat-card/mat-card-footer/div/pd-button-primary/button/span[4]", "Clicou em Salvar"):
            return False
        time.sleep(0.8)
        try:
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                time.sleep(0.25)
                print("✓ Fechou a aba e voltou para a principal")
        except:
            print("✗ Erro ao fechar aba")
            return False
        print("\n✓ Anotação realizada com sucesso!")
        return True
    except Exception as e:
        print("\n✗ Erro durante o processo de anotação")
        try:
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
        except:
            pass
        return False

In [7]:
# Célula 7 – Função para Selecionar Demandas Analisadas e Concluir
def selecionar_demandas_analisadas_e_concluir(driver):
    try:
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(1)
        agrupamento_xpath = "//span[contains(text(), 'Intimação - Trânsito em julgado')]/ancestor::pd-demanda-agrupada"
        agrupamento = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, agrupamento_xpath))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", agrupamento)
        time.sleep(2)
        demandas = driver.find_elements(By.CSS_SELECTOR, "pd-demanda-mesa pd-demanda")
        contador_analisadas = 0
        for demanda in demandas:
            if "Autos já analisados - GCA" in demanda.text:
                checkbox = demanda.find_element(By.CSS_SELECTOR, "input[type='checkbox'].mdc-checkbox__native-control")
                driver.execute_script("arguments[0].click();", checkbox)
                contador_analisadas += 1
                time.sleep(0.2)
        print(f"Foram selecionadas {contador_analisadas} demandas analisadas.")
        if contador_analisadas > 0:
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
            driver.execute_script("arguments[0].scrollIntoView(true);", agrupamento)
            time.sleep(1)
            concluir_xpath = "//pd-demanda-agrupada-actions-container//button[.//mat-icon[@data-mat-icon-name='concluir']]"
            concluir_button = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, concluir_xpath))
            )
            driver.execute_script("arguments[0].click();", concluir_button)
            time.sleep(1)
            modal_concluir_xpath = "//pd-modal-container//pd-button-primary/button"
            modal_concluir = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, modal_concluir_xpath))
            )
            driver.execute_script("arguments[0].click();", modal_concluir)
            print("✓ Intimações concluídas com sucesso!")
    except Exception as e:
        print(f"Erro durante o processo: {str(e)}")

In [8]:
# Célula 8 – Função para Verificar o Grupamento e Processar o Lote
def verificar_grupamento_visivel(driver):
    try:
        grupamento = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//pd-demanda-agrupada[contains(.,'Intimação - Trânsito em julgado')]"))
        )
        intimacoes = grupamento.find_elements(By.CSS_SELECTOR, "pd-demanda-mesa pd-demanda")
        if not intimacoes:
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", grupamento)
            time.sleep(1)
            try:
                grupamento.click()
                time.sleep(1)
                nao_alocados = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.XPATH, "//span[contains(text(), 'Não alocados')]"))
                )
                driver.execute_script("arguments[0].click();", nao_alocados)
                time.sleep(1)
            except:
                return False
        return True
    except Exception as e:
        print("× Erro ao verificar grupamento")
        return False

def retornar_aba_principal(driver):
    """Função para garantir que o driver está na aba principal"""
    try:
        if len(driver.window_handles) > 1:
            print("Retornando para aba principal...")
            # Fechar todas as abas exceto a principal
            main_window = driver.window_handles[0]
            for handle in driver.window_handles[1:]:
                driver.switch_to.window(handle)
                driver.close()
            driver.switch_to.window(main_window)
            print("✓ Retorno à aba principal concluído")
            return True
        return True  # Já está na aba principal
    except Exception as e:
        print(f"× Erro ao retornar à aba principal: {str(e)}")
        return False

def recuperar_fluxo(driver):
    """Função para tentar recuperar o fluxo reiniciando a partir do grupamento"""
    try:
        print("\n>> Tentando recuperar fluxo...")
        retornar_aba_principal(driver)
        
        # Atualizar a página e esperar carregar
        print("Atualizando página...")
        driver.refresh()
        time.sleep(3)
        
        # Buscar grupamento de intimações novamente
        print("Buscando grupamento de intimações...")
        grupamento = encontrar_intimacao_transito(driver)
        if grupamento:
            print("✓ Fluxo recuperado com sucesso")
            time.sleep(1)
            return True
        else:
            print("× Falha ao recuperar fluxo - grupamento não encontrado")
            return False
    except Exception as e:
        print(f"× Erro durante recuperação de fluxo: {str(e)}")
        return False

def processar_lote_intimacoes(driver, shared_processadas=None, lock=None):
    if shared_processadas is None:
        shared_processadas = set()
    total_verificadas = 0
    total_processadas = 0
    contador_para_concluir = 0
    falhas_consecutivas = 0
    max_falhas_permitidas = 3
    
    print("\n>> Iniciando processamento de intimações...")
    
    while True:
        try:
            # Verificar se estamos na aba principal e se o grupamento está visível
            if not retornar_aba_principal(driver) or not verificar_grupamento_visivel(driver):
                print("× Recuperando grupamento...")
                if not recuperar_fluxo(driver):
                    falhas_consecutivas += 1
                    if falhas_consecutivas >= max_falhas_permitidas:
                        print(f"× {max_falhas_permitidas} falhas consecutivas. Tentando reiniciar completamente...")
                        falhas_consecutivas = 0
                        driver.get("https://pgeweb.sp.gov.br/app/mesa")
                        time.sleep(5)
                        if not recuperar_fluxo(driver):
                            print("× Falha crítica na recuperação. Tentando novamente...")
                            time.sleep(10)
                            continue
                time.sleep(1)
                continue
            
            # Resetar contador de falhas quando tudo estiver ok
            falhas_consecutivas = 0
            
            # Processar próxima intimação
            if lock:
                with lock:
                    sucesso, shared_processadas = clicar_primeira_intimacao_nao_processada(driver, shared_processadas)
            else:
                sucesso, shared_processadas = clicar_primeira_intimacao_nao_processada(driver, shared_processadas)
                
            if not sucesso:
                if contador_para_concluir > 0:
                    print("\n>> Concluindo intimações restantes...")
                    try:
                        selecionar_demandas_analisadas_e_concluir(driver)
                        print("✓ Lote concluído")
                        time.sleep(5)
                        
                        # Reiniciar o processo buscando novamente o grupamento
                        print("\n>> Reiniciando processo para buscar novas intimações...")
                        driver.refresh()
                        time.sleep(5)
                        
                        # Reiniciar contadores para o novo lote
                        total_verificadas = 0
                        total_processadas = 0
                        contador_para_concluir = 0
                        
                        # Buscar grupamento novamente como na célula 2
                        encontrar_intimacao_transito(driver)
                        print("✓ Processo reiniciado com sucesso")
                        time.sleep(2)
                        continue
                    except Exception as concluir_erro:
                        print(f"× Erro ao concluir lote: {str(concluir_erro)}")
                        # Mesmo com erro, tenta reiniciar o processo
                        recuperar_fluxo(driver)
                        continue
                else:
                    # Se não há intimações e nenhuma foi processada, aguarda um tempo e tenta novamente
                    print("Nenhuma intimação disponível para processar. Aguardando...")
                    time.sleep(30)  # Aguarda 30 segundos antes de verificar novamente
                    driver.refresh()
                    time.sleep(5)
                    encontrar_intimacao_transito(driver)
                    continue
                
            total_verificadas += 1
            print(f"\n>> Intimação #{total_verificadas}")
            
            try:
                if exibir_processos(driver):
                    time.sleep(2)
                    if processar_intimacoes(driver, shared_processadas):
                        if anota_ja_analis(driver):
                            total_processadas += 1
                            contador_para_concluir += 1
                            print(f"✓ Intimação processada ({contador_para_concluir}/20)")                        
            except Exception as process_error:
                print(f"× Erro ao processar intimação: {str(process_error)}")
                retornar_aba_principal(driver)
                continue
                
            # Atualizar página periodicamente para evitar timeout da sessão
            if total_verificadas % 30 == 0:
                print("\n>> Atualizando página...")
                if not recuperar_fluxo(driver):
                    print("× Falha ao atualizar página")
            
            time.sleep(1)
            
        except Exception as e:
            print(f"\n× Erro geral durante processamento: {str(e)}")
            falhas_consecutivas += 1
            print(f"Falhas consecutivas: {falhas_consecutivas}/{max_falhas_permitidas}")
            
            if not retornar_aba_principal(driver):
                print("× Não foi possível retornar à aba principal")
                
            if falhas_consecutivas >= max_falhas_permitidas:
                print("\n>> Muitas falhas consecutivas, tentando recuperação completa...")
                if recuperar_fluxo(driver):
                    falhas_consecutivas = 0
                else:
                    print("× Falha na recuperação completa")
            
            time.sleep(2)
    
    # Este código nunca será alcançado no loop infinito, mas mantido para compatibilidade
    print(f"\n=== Processamento finalizado ===")
    print(f"Total verificadas: {total_verificadas}")
    print(f"Total processadas: {total_processadas}")

    return shared_processadas, total_processadas, total_verificadas

In [ ]:
# Célula 9 – Execução Paralela (5 Instâncias)
from threading import Lock
import concurrent.futures

# Variável global compartilhada e lock para evitar processamento duplicado
processadas_global = set()
processadas_lock = Lock()

def create_driver():
    driver = iniciar_navegador()
    if driver:
        driver = login_attus(driver)
    return driver

def run_processor(instance_id):
    driver = create_driver()
    if not driver:
        return None
    try:
        # processar_lote_intimacoes deve ter sido ajustada para emitir apenas os seguintes logs:
        #   >> Intimação #    (para cada intimação processada)
        #   Buscando siglas no histórico...
        #   Sigla encontrada ou Sigla não encontrada
        #   ✓ Anotação realizada com sucesso!
        #   ✓ Intimação processada
        resultados = processar_lote_intimacoes(driver, processadas_global, processadas_lock)
        return resultados
    except Exception as e:
        return None
    finally:
        driver.quit()

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    future_to_instance = {executor.submit(run_processor, i): i for i in range(1,6)}
    for future in concurrent.futures.as_completed(future_to_instance):
        res = future.result()
        if res:
            shared, total_processadas, total_verificadas = res
            print(f">> Intimação #{total_verificadas}")
            if total_processadas:
                print("✓ Intimação processada")
        else:
            print("Nenhuma intimação processada")

Usando ChromeDriver em: C:\Users\gusta\.wdm\drivers\chromedriver\win64\135.0.7049.42\chromedriver-win32\chromedriver.exe
Usando ChromeDriver em: C:\Users\gusta\.wdm\drivers\chromedriver\win64\135.0.7049.42\chromedriver-win32\chromedriver.exe
Usando ChromeDriver em: C:\Users\gusta\.wdm\drivers\chromedriver\win64\135.0.7049.42\chromedriver-win32\chromedriver.exe
Usando ChromeDriver em: C:\Users\gusta\.wdm\drivers\chromedriver\win64\135.0.7049.42\chromedriver-win32\chromedriver.exe
Usando ChromeDriver em: C:\Users\gusta\.wdm\drivers\chromedriver\win64\135.0.7049.42\chromedriver-win32\chromedriver.exe
Acessando o site do Attus...
Acessando o site do Attus...
Esperando pela tela de login...
Esperando pela tela de login...
Acessando o site do Attus...
Acessando o site do Attus...
Acessando o site do Attus...
Esperando pela tela de login...
Esperando pela tela de login...
Campo de email encontrado, inserindo email...
Campo de email encontrado, inserindo email...
Esperando pela tela de login..